In [3]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [1]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k = None, device = 0)
classifier("I love this!")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:  13%|#2        | 41.9M/329M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/j-hartmann/emotion-english-distilroberta-base/resolve/main/pytorch_model.bin: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:  89%|########9 | 294M/329M [00:00<?, ?B/s]

D:\Book-Recommendation-System\Book_Recommender_venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danie\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Falling back to torch.float32 because loading with the original dtype f

...

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611991785466671},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [4]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

In [5]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.729602038860321},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816231459379196},
  {'label': 'joy', 'score': 0.0479426383972168},
  {'label': 'anger', 'score': 0.009156374260783195},
  {'label': 'disgust', 'score': 0.002628476358950138},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937053322792053},
  {'label': 'disgust', 'score': 0.2735915184020996},
  {'label': 'joy', 'score': 0.10908309370279312},
  {'label': 'sadness', 'score': 0.0936274453997612},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.242733433842659},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.01421149075031

In [6]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

isbn = []

emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key= lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
        return {label: np.max(scores) for label, scores in per_emotion_scores.items()}
        
        

In [8]:
from tqdm import tqdm

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [08:00<00:00, 10.82it/s]


In [9]:
emotions_dataFrame = pd.DataFrame(emotion_scores)
emotions_dataFrame["isbn13"] = isbn


In [10]:
emotions_dataFrame

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.009156,0.002628,0.068162,0.047943,0.140386,0.002122,0.729602,9780002005883
1,0.005967,0.002887,0.003810,0.704422,0.217759,0.004509,0.060646,9780002261982
2,0.041301,0.024568,0.104061,0.767238,0.042176,0.010860,0.009796,9780006178736
3,0.016036,0.060695,0.001692,0.161758,0.732685,0.020988,0.006147,9780006280897
4,0.013624,0.122243,0.095043,0.008336,0.272614,0.475880,0.012260,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.025156,0.001939,0.094667,0.002254,0.010511,0.857256,0.008218,9788172235222
5193,0.005602,0.003775,0.018216,0.400263,0.338892,0.005487,0.227765,9788173031014
5194,0.008463,0.009146,0.013295,0.620453,0.329753,0.010788,0.008101,9788179921623
5195,0.005475,0.034544,0.003970,0.258354,0.648010,0.017372,0.032275,9788185300535


In [12]:
books = pd.merge(books, emotions_dataFrame, on = "isbn13")

In [13]:
books.to_csv("books_with_emotions", index = False)